In [1]:
import numpy as np
import pandas as pd 

In [64]:
data = pd.read_excel("/home/bennour/Studies/TBS Courses/BO/refinery_data.xlsx")

In [65]:
data

,Producer,un_gasoline,fuel_oil,naphta,by_products,refining_cost,purchase_price,max_barrels,min_barrels
0,Nigeria,0.22,0.20,0.05,0.53,4.00,24.0,99999999.0,75000.0
1,Gabon,0.18,0.22,0.12,0.48,4.25,25.0,200000.0,60000.0
2,Venzuella,0.25,0.05,0.22,0.48,4.15,22.3,99999999.0,50000.0
3,Mexico,0.20,0.26,0.14,0.40,4.50,24.2,250000.0,50000.0
4,demand,200000.00,100000.00,40000.00,0.00,NaN,NaN,NaN,NaN
5,selling_price,200.00,125.00,310.00,20.00,NaN,NaN,NaN,NaN


In [66]:
from pulp import *

In [67]:
prob = LpProblem("refinery", LpMaximize)

In [119]:
# create a list of producers
producers = list(data.Producer[:4])
# create a list of by_products 
products = list(data.iloc[:,1:5])

In [122]:
# create a dictionary of refining costs
RC = dict(zip(producers, data.refining_cost[:4]))
# create a dictionary of purchase price
PP = dict(zip(producers, data.purchase_price[:4]))
# create a dictionay of selling price 
SP = dict(zip(products, data.iloc[5, 1:5]))
# dictionary for the demands 
D = dict(zip(products, data.iloc[4, 1:5]))
# dictionary for minimum barrels
m = dict(zip(producers, data.min_barrels[:4]))
# dictionary for maximum barrels
M = dict(zip(producers, data.max_barrels[:4]))

In [138]:
# create a dictionary of propotions marginal profit per producer
prop_nigeria = dict(zip(by_products, data.iloc[0, 1:5]))
prop_gabon = dict(zip(by_products, data.iloc[1, 1:5]))
prop_venzuella = dict(zip(by_products, data.iloc[2, 1:5]))
prop_mexico = dict(zip(by_products, data.iloc[3, 1:5]))

# create a dictionary of proportions marginal profit per by_product 
prop_gas = dict(zip(producers, data.iloc[0:4, 1]))
prop_oil = dict(zip(producers, data.iloc[0:4, 2]))
prop_naf = dict(zip(producers, data.iloc[0:4, 3]))
prop_byp = dict(zip(producers, data.iloc[0:4, 4]))

In [139]:
prop = {
    'Nigeria': prop_nigeria,
    'Gabon': prop_gabon,
    'Venzuella': prop_venzuella,
    'Mexico': prop_mexico
}

In [145]:
capacity = 900000

In [125]:
x = LpVariable.dicts("Producer", producers, lowBound=0, cat = 'integer')

In [126]:
# main objective function:
# the profit after buying, refining, and selling, the barrels from each producer
prob += lpSum([x[i]*SP[j]*prop[i][j] - x[i]*RC[i]*PP[i] for i in producers for j in products])

/home/bennour/miniconda3/envs/main/lib/python3.7/site-packages/pulp/pulp.py:1348: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [146]:
# adding constraints to the prob model

prob += lpSum([prop['Nigeria'][j] for j in products]) == 1
prob += lpSum([prop['Gabon'][j] for j in products]) == 1
prob += lpSum([prop['Venzuella'][j] for j in products]) == 1
prob += lpSum([prop['Mexico'][j] for j in products]) == 1
prob += lpSum([x[i]*prop[i][j] for i in producers for j in products]) <= {D[j] for j in products} 
prob += lpSum([x[i] for i in producers]) <= capacity
prob += {x[i] for i in producers} >= {m[i] for i in producers}
prob += {x[i] for i in producers} <= {M[i] for i in producers}

/home/bennour/miniconda3/envs/main/lib/python3.7/site-packages/pulp/pulp.py:1353: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [148]:
LpStatus[prob.status]

'Infeasible'